In [138]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [139]:
import sys
sys.path.insert(0, '..//')
import pandas as pd
import numpy as np
import seaborn as sns

from pathlib import Path

from src.common import schema, enums, functions


In [140]:
path_store_results = Path(r"C:\Users\sceac10\OneDrive - Cardiff University\04 - Projects\22 - Heat demand scotland\data")

path_census_data = Path(r"C:\Users\sceac10\OneDrive - Cardiff University\04 - Projects\22 - Heat demand scotland\data\CT_0043_2011.xlsx")
sheet_name = "CT_0043_2011"

In [141]:
census_data = pd.read_excel(path_census_data, sheet_name=sheet_name, header=4, na_values='-')

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\sceac10\\OneDrive - Cardiff University\\04 - Projects\\22 - Heat demand scotland\\data\\CT_0043_2011.xlsx'

In [ ]:
list_oas = [c for c in census_data['Unnamed: 0'].unique() if str(c)[0]=='S' and 'Scotland' not in c]
list_oas

['S00088956',
 'S00088957',
 'S00088958',
 'S00088959',
 'S00088960',
 'S00088961',
 'S00088962',
 'S00088963',
 'S00088964',
 'S00088965',
 'S00088966',
 'S00088967',
 'S00088968',
 'S00088969',
 'S00088970',
 'S00088971',
 'S00088972',
 'S00088973',
 'S00088974',
 'S00088975',
 'S00088976',
 'S00088977',
 'S00088978',
 'S00088979',
 'S00088980',
 'S00088981',
 'S00088982',
 'S00088983',
 'S00088984',
 'S00088985',
 'S00088986',
 'S00088987',
 'S00088988',
 'S00088989',
 'S00088990',
 'S00088991',
 'S00088992',
 'S00088993',
 'S00088994',
 'S00088995',
 'S00088996',
 'S00088997',
 'S00088998',
 'S00088999',
 'S00089000',
 'S00089001',
 'S00089002',
 'S00089003',
 'S00089004',
 'S00089005',
 'S00089006',
 'S00089007',
 'S00089008',
 'S00089009',
 'S00089010',
 'S00089011',
 'S00089012',
 'S00089013',
 'S00089014',
 'S00089015',
 'S00089016',
 'S00089017',
 'S00089018',
 'S00089019',
 'S00089020',
 'S00089021',
 'S00089022',
 'S00089023',
 'S00089024',
 'S00089025',
 'S00089026',
 'S000

In [ ]:
def extract_census_data(dataf:pd.DataFrame, temp_oa_code:str)->pd.DataFrame:
  """Extract the census data related to the OA code."""
  start_idx =dataf.loc[dataf['Unnamed: 0']==temp_oa_code].index[0]+1
  end_idx = start_idx+7
  temp_df = dataf.loc[start_idx:end_idx].copy()
  return temp_df.set_index(['Unnamed: 0'])


In [ ]:
def transform_census_data(dataf:pd.DataFrame)->pd.DataFrame:
  """Transform census data into a finalCensusSchema format."""

  cols_to_keep = [schema.censusSchema.detached, schema.censusSchema.semi_detached, schema.censusSchema.terraced,schema.censusSchema.flat]
  index_to_keep = [schema.censusSchema.electric_heating, schema.censusSchema.gas_heating, schema.censusSchema.oil_heating, schema.censusSchema.solid_fuel_heating]
  dataf = dataf.loc[dataf.index.isin(index_to_keep), cols_to_keep]

  rename_cols = {
    schema.censusSchema.detached:enums.dwelling_types.DETACHED.value, 
    schema.censusSchema.semi_detached:enums.dwelling_types.SEMIDETACHED.value, 
    schema.censusSchema.terraced:enums.dwelling_types.TERRACED.value, 
    schema.censusSchema.flat:enums.dwelling_types.FLAT.value, 
    }
  rename_idx = {
    schema.censusSchema.electric_heating:enums.heating_systems.RESISTANCE.value, 
    schema.censusSchema.gas_heating:enums.heating_systems.GASBOILER.value,
    schema.censusSchema.oil_heating:enums.heating_systems.OILBOILER.value, 
    schema.censusSchema.solid_fuel_heating:enums.heating_systems.SOLIDFUELBOILER.value,
    schema.censusSchema.other:enums.heating_systems.UNCATEGORIZED.value
    }

  dataf.rename(columns=rename_cols, index=rename_idx, inplace=True)

  dataf = dataf.unstack().to_frame().reset_index()
  dataf.columns = [schema.finalCensusSchema.dwelling_type, schema.finalCensusSchema.heating_system, schema.finalCensusSchema.number]
  dataf.fillna(0, inplace=True)
  
  return dataf


def assign_oa_code(dataf:pd.DataFrame, oa_code:str)->pd.DataFrame:
  dataf[schema.finalCensusSchema.oa_code] = oa_code
  return dataf

In [ ]:
## Extract and transform census data
frames = []
for temp_oa in list_oas:
  print(temp_oa)
  temp_df = census_data.pipe(extract_census_data, temp_oa).pipe(transform_census_data).pipe(assign_oa_code, temp_oa)
  if len(temp_df)>0:
    frames.append(temp_df)

formatted_census_data = pd.concat(frames, axis=0)
formatted_census_data.to_csv(path_store_results/"formatted_census_data.csv")

S00088956
S00088957
S00088958
S00088959
S00088960
S00088961
S00088962
S00088963
S00088964
S00088965
S00088966
S00088967
S00088968
S00088969
S00088970
S00088971
S00088972
S00088973
S00088974
S00088975
S00088976
S00088977
S00088978
S00088979
S00088980
S00088981
S00088982
S00088983
S00088984
S00088985
S00088986
S00088987
S00088988
S00088989
S00088990
S00088991
S00088992
S00088993
S00088994
S00088995
S00088996
S00088997
S00088998
S00088999
S00089000
S00089001
S00089002
S00089003
S00089004
S00089005
S00089006
S00089007
S00089008
S00089009
S00089010
S00089011
S00089012
S00089013
S00089014
S00089015
S00089016
S00089017
S00089018
S00089019
S00089020
S00089021
S00089022
S00089023
S00089024
S00089025
S00089026
S00089027
S00089028
S00089029
S00089030
S00089031
S00089032
S00089033
S00089034
S00089035
S00089036
S00089037
S00089038
S00089039
S00089040
S00089041
S00089042
S00089043
S00089044
S00089045
S00089046
S00089047
S00089048
S00089049
S00089050
S00089051
S00089052
S00089053
S00089054
S00089055


In [ ]:
formatted_census_data = pd.read_csv(path_store_results/"formatted_census_data.csv", index_col=0)

In [ ]:
formatted_census_data.head()

,Dwelling type,Heating system,Number of dwellings,OA_code
0,detached,gas boiler,62.0,S00088956
1,detached,resistance,0.0,S00088956
2,detached,oil boiler,2.0,S00088956
3,detached,solid fuel boiler,0.0,S00088956
4,semi-detached,gas boiler,6.0,S00088956


In [ ]:
geo_code_lookup = functions.get_postcode_lsoa_lookup("")
columns_to_keep = [
    schema.geoLookupSchema.oa,
    schema.geoLookupSchema.lsoa, schema.geoLookupSchema.msoa,
    schema.geoLookupSchema.ladcd, schema.geoLookupSchema.ladnm
]
geo_code_lookup = geo_code_lookup[cols_to_keep].drop_duplicates().copy()


In [ ]:
formatted_census_data

,Dwelling type,Heating system,Number of dwellings,OA_code,oa11cd,lsoa11cd,msoa11cd,ladcd,ladnm
0,detached,gas boiler,62.0,S00088956,S00088956,S01006732,S02001275,S12000033,aberdeen city
1,detached,resistance,0.0,S00088956,S00088956,S01006732,S02001275,S12000033,aberdeen city
2,detached,oil boiler,2.0,S00088956,S00088956,S01006732,S02001275,S12000033,aberdeen city
3,detached,solid fuel boiler,0.0,S00088956,S00088956,S01006732,S02001275,S12000033,aberdeen city
4,semi-detached,gas boiler,6.0,S00088956,S00088956,S01006732,S02001275,S12000033,aberdeen city
...,...,...,...,...,...,...,...,...,...
737995,terraced,solid fuel boiler,0.0,S00135306,S00135306,S01009023,S02001689,S12000013,na heileanan siar
737996,flat,gas boiler,0.0,S00135306,S00135306,S01009023,S02001689,S12000013,na heileanan siar
737997,flat,resistance,0.0,S00135306,S00135306,S01009023,S02001689,S12000013,na heileanan siar
737998,flat,oil boiler,0.0,S00135306,S00135306,S01009023,S02001689,S12000013,na heileanan siar


In [ ]:
formatted_census_data = formatted_census_data.merge(geo_code_lookup, left_on=schema.finalCensusSchema.oa_code, right_on=schema.geoLookupSchema.oa)
census_data_lsoa_level = formatted_census_data.groupby([schema.geoLookupSchema.lsoa, schema.finalCensusSchema.dwelling_type, schema.finalCensusSchema.heating_system]).agg({schema.finalCensusSchema.number:'sum'})
census_data_lsoa_level = census_data_lsoa_level.reset_index()
census_data_lsoa_level[schema.finalCensusSchema.dwelling_category] = census_data_lsoa_level.apply(lambda x: functions.get_dwelling_category(x[schema.finalCensusSchema.dwelling_type], x[schema.finalCensusSchema.heating_system]), axis=1)
census_data_lsoa_level.to_csv(path_store_results/"formatted_census_data_lsoa_level.csv")
census_data_lsoa_level.head()

,lsoa11cd,Dwelling type,Heating system,Number of dwellings,Dwelling category
0,S01006506,detached,gas boiler,85.0,detached gas boiler
1,S01006506,detached,oil boiler,10.0,detached oil boiler
2,S01006506,detached,resistance,0.0,detached resistance
3,S01006506,detached,solid fuel boiler,1.0,detached solid fuel boiler
4,S01006506,flat,gas boiler,158.0,flat gas boiler
